In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# MODEL ARCHITECTURE

In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class SupervisedAttentionModule(nn.Module):
    def __init__(self,num_features ):
        super(SupervisedAttentionModule, self).__init__()
        self.conv1 = nn.Conv2d(num_features, 3, 1)
        self.conv2 = nn.Conv2d(num_features, num_features, 1)
        self.conv3 = nn.Conv2d(3, num_features, 1)
    def forward(self, f_in, degraded_1):
        r_s = self.conv1(f_in)
        x_s = r_s + degraded_1
        f_out = self.conv2(f_in)
        m = torch.sigmoid(self.conv3(x_s))
        f_out = f_out * m
        f_out = f_out + f_in
        return f_out, x_s
    
class ChannelAttentionBlock(nn.Module):
    def __init__(self, num_features, kernel, reduction):
        super(ChannelAttentionBlock, self).__init__()
        self.prelu = nn.PReLU()
        self.conv0 = nn.Conv2d(num_features, num_features, kernel, padding = 'same')
        self.conv1 = nn.Conv2d(num_features, num_features, kernel, padding = 'same')
        self.conv2 = nn.Conv2d(num_features, num_features//reduction, 1)
        self.conv3 = nn.Conv2d(num_features//reduction, num_features, 1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.global_avg_pooling = nn.AdaptiveAvgPool2d(1)
    
    def forward(self, x):
        x1 = self.conv0(x)
        #print(x1.shape)
        x1 = self.prelu(x1)
        x1 = self.conv1(x1)
        #print(x1.shape)
        x2 = self.global_avg_pooling(x1)
        #print(x2.shape)
        x2 = self.conv2(x2)
        #print(x2.shape)
        x2 = self.prelu(x2)
        x2 = self.conv3(x2)
        #print(x2.shape)
        x2 = self.sigmoid(x2)
        x2 = x2*x1
        #print(x2.shape)
        #print(x.shape)
        out = x2+x
        return out
        
        

class OriginalResolutionBlock(nn.Module):
    def __init__(self, kernel, reduction, num_cabs = 8, num_features = 96):
        super(OriginalResolutionBlock, self).__init__()
        self.cab_list = []
        for i in range(num_cabs):
            self.cab_list.append(ChannelAttentionBlock(num_features,kernel,reduction))
        self.cab_list.append(nn.Conv2d(num_features,num_features,kernel, padding = 'same'))
        self.orb = nn.Sequential(*self.cab_list)
    
    def forward(self,x):
        out = self.orb(x)
        out += x
        return out
        
        
    

class OriginalResolutionSubNetwork(nn.Module):
    def __init__(self, num_features, kernel_size, reduction, num_cabs):
        super(OriginalResolutionSubNetwork, self).__init__()
        self.orb1 = OriginalResolutionBlock(kernel_size, reduction, num_cabs, num_features)
        self.orb2 = OriginalResolutionBlock(kernel_size, reduction, num_cabs, num_features)
        self.orb3 = OriginalResolutionBlock(kernel_size, reduction, num_cabs, num_features)
        
        
        self.csffenc1 = nn.Conv2d(num_features, num_features, 1)
        self.csffenc2 = nn.Conv2d(num_features + (num_features//reduction), num_features, 1)
        self.csffenc3 = nn.Conv2d(num_features + 2*(num_features//reduction), num_features, 1)
         
        self.csffdec1 = nn.Conv2d(num_features, num_features, 1)
        self.csffdec2= nn.Conv2d(num_features, num_features, 1)
        self.csffdec3= nn.Conv2d(num_features+ (num_features//reduction), num_features, 1)
        
        self.up1 = nn.Upsample(scale_factor= 2, mode = 'bilinear', align_corners=True)
        self.up2 = nn.Upsample(scale_factor = 4, mode = 'bilinear', align_corners = True)
        
    def forward(self, x , e1, d1, e2, d2, e3, d3):
        
        #print(x.shape, e1.shape, d1.shape, e2.shape, d2.shape, e3.shape, d3.shape)
        
        x = self.orb1(x)
        #print(f"Shape of e1 is {self.csffenc1(e1).shape}, Shape of d1 is {self.csffdec1(d1).shape}, Shape of x is {x.shape}")
        x = x + self.csffenc1(e1) + self.csffdec1(d1)
        x = self.orb2(x)
        #print(f"Shape of e2 is {self.csffenc2(e2).shape}, Shape of d2 is {self.csffdec2(d2).shape}, Shape of x is {x.shape}")
        x = x + self.up1(self.csffenc2(e2)) + self.up1(self.csffdec2(d2))
        x = self.orb3(x)
        #print(f"Shape of e3 is {self.csffenc3(e3).shape}, Shape of d3 is {self.csffdec3(d3).shape}, Shape of x is {x.shape}")
        x = x + self.up2(self.csffenc3(e3)) + self.up2(self.csffdec3(d3))
        return x

same = 'same'
class Encoder(nn.Module):
    def __init__(self, num_features, kernel, reduction, recieve_csff_ip = False):
        super(Encoder, self).__init__()
        self.encoder_level1 = nn.Sequential(
            nn.Conv2d(num_features, num_features, kernel, padding = same),
            nn.ReLU(),
            nn.Conv2d(num_features, num_features, kernel, padding = same),
            nn.ReLU()
        )
        self.encoder_level2 = nn.Sequential(
            nn.Conv2d(num_features, num_features + num_features // reduction, kernel, padding = same),
            nn.ReLU(),
            nn.Conv2d(num_features + num_features // reduction, num_features + num_features // reduction, kernel, padding = same),
            nn.ReLU()
        )
        self.encoder_level3 = nn.Sequential(
            nn.Conv2d(num_features + num_features // reduction, num_features + 2 * (num_features // reduction), kernel, padding = same),
            nn.ReLU(),
            nn.Conv2d(num_features + 2 * (num_features // reduction), num_features + 2 * (num_features // reduction), kernel, padding = same),
            nn.ReLU()
        )
        self.down12 = nn.Upsample(scale_factor=0.5, mode='bilinear', align_corners=True)
        self.down23 = nn.Upsample(scale_factor=0.5, mode='bilinear', align_corners=True)
        
        if recieve_csff_ip:
            self.csff1_1 = nn.Conv2d(num_features, num_features, 1)
            self.csff1_2 = nn.Conv2d(num_features, num_features, 1)
            self.csff2_1 = nn.Conv2d(num_features + num_features // reduction, num_features + num_features // reduction, 1)
            self.csff2_2 = nn.Conv2d(num_features , num_features + num_features // reduction, 1)
            self.csff3_1 = nn.Conv2d( num_features + 2 * (num_features // reduction),  num_features + 2 * (num_features // reduction), 1)
            self.csff3_2 = nn.Conv2d( num_features + 1 * (num_features // reduction),  num_features + 2 * (num_features // reduction), 1)


    def forward(self, x, csff_input1_1 = None, csff_input1_2 = None, csff_input2_1 = None, csff_input2_2 = None, csff_input3_1 = None, csff_input3_2 = None):
        
        enc1 = self.encoder_level1(x)
        if csff_input1_1 is not None and csff_input1_2 is not None:
            #print(f"Shape of csff_input1_1 is {csff_input1_1.shape}, csff_input1_2 is {csff_input1_2.shape}")
            enc1 = enc1 + self.csff1_1(csff_input1_1) + self.csff1_2(csff_input1_2)
            
        x = self.down12(enc1)
        
        enc2 = self.encoder_level2(x)
        if csff_input2_1 is not None and csff_input2_2 is not None:
            #print(f"Shape of csff_input2_1 is {csff_input2_1.shape}, csff_input2_2 is {csff_input2_2.shape}")
            enc2 = enc2 + self.csff2_1(csff_input2_1) + self.csff2_2(csff_input2_2)
        
        x = self.down23(enc2)
        enc3 = self.encoder_level3(x)
        if csff_input3_1 is not None and csff_input3_2 is not None:
            enc3 = enc3 + self.csff3_1(csff_input3_1) + self.csff3_2(csff_input3_2)
            
        return [enc1, enc2, enc3]


class Decoder(nn.Module):
    def __init__(self, num_features, kernel, reduction):
        super(Decoder, self).__init__()
        self.decoder_level1 = nn.Sequential(
            nn.Conv2d(num_features, num_features, kernel, padding = same),
            nn.ReLU(),
            nn.Conv2d(num_features, num_features, kernel, padding = same),
            nn.ReLU()
        )
        self.decoder_level2 = nn.Sequential(
            nn.Conv2d(num_features + num_features // reduction, num_features + num_features // reduction, kernel, padding = same),
            nn.ReLU(),
            nn.Conv2d(num_features + num_features // reduction, num_features, kernel, padding = same),
            nn.ReLU()
        )
        self.decoder_level3 = nn.Sequential(
            nn.Conv2d(num_features + 2*(num_features // reduction), num_features + 2*(num_features // reduction), kernel, padding = same),
            nn.ReLU(),
            nn.Conv2d(num_features + 2*(num_features // reduction), num_features + 1*(num_features // reduction), kernel, padding = same),
            nn.ReLU()
        )
        self.up32 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.up21 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

    def forward(self, enc1, enc2,enc3):
        dec3 = self.decoder_level3(enc3)
        x3 = self.up32(dec3)+enc2
        dec2 = self.decoder_level2(x3)
        x2 = self.up21(dec2)+enc1
        dec1 = self.decoder_level1(x2)
        return [dec1, dec2, dec3]
    


class MPRNET(nn.Module):
    def __init__(self, num_features = 80, kernel_size = 3, reduction =4 , num_cabs = 8 ):
        super(MPRNET, self).__init__()
        self.conv_stage1 = nn.Conv2d(3, num_features, kernel_size, padding = 'same')
        self.conv_stage2 = nn.Conv2d(3, num_features, kernel_size, padding = 'same')
        self.conv_stage3 = nn.Conv2d(3, num_features, kernel_size, padding = 'same')
        self.final_conv = nn.Conv2d(num_features, 3, kernel_size, padding = 'same')
        
        
        self.sam_stage1 = SupervisedAttentionModule(num_features)
        self.sam_stage2 = SupervisedAttentionModule(num_features)
        
        self.cab_stage1 = ChannelAttentionBlock(num_features, kernel_size, reduction)
        self.cab_stage2 = ChannelAttentionBlock(num_features, kernel_size, reduction)
        self.cab_stage3 = ChannelAttentionBlock(num_features, kernel_size, reduction)
        
        self.encoder1 = Encoder(num_features, kernel_size, reduction)
        self.encoder2 = Encoder(num_features, kernel_size, reduction, True)
        
        self.decoder1 = Decoder(num_features, kernel_size, reduction)
        self.decoder2 = Decoder(num_features, kernel_size, reduction)
        
        self.ors_net = OriginalResolutionSubNetwork(num_features, kernel_size, reduction, num_cabs)
        
    def forward(self,image):
        
        height = image.shape[-2]
        width = image.shape[-1]
        
        left_image = image[:,:,:,0:int(width/2)]
        right_image = image[:,:,:,int(width/2):width]
        
        top_left_image = left_image[:,:,0:int(height/2),:]
        bottom_left_image = left_image[:,:,int(height/2):height,:]
        top_right_image = right_image[:,:,0:int(height/2),:]
        bottom_right_image= right_image[:,:,int(height/2):height,:]
        
        x1_tl = self.conv_stage1(top_left_image)
        x1_tl = self.cab_stage1(x1_tl)
        enc1_stage1_tl, enc2_stage1_tl, enc3_stage1_tl = self.encoder1(x1_tl)
        
        
        x1_bl = self.conv_stage1(bottom_left_image)
        x1_bl = self.cab_stage1(x1_bl)
        enc1_stage1_bl, enc2_stage1_bl, enc3_stage1_bl = self.encoder1(x1_bl)
        
        enc1_stage1_left, enc2_stage1_left, enc3_stage1_left = torch.cat((enc1_stage1_tl,enc1_stage1_bl), dim = 2), torch.cat((enc2_stage1_tl,enc2_stage1_bl), dim = 2), torch.cat((enc3_stage1_tl,enc3_stage1_bl), dim = 2)
        
        x1_tr = self.conv_stage1(top_right_image)
        x1_tr = self.cab_stage1(x1_tr)
        enc1_stage1_tr, enc2_stage1_tr, enc3_stage1_tr = self.encoder1(x1_tr)
        
        x1_br = self.conv_stage1(bottom_right_image)
        x1_br = self.cab_stage1(x1_br)
        enc1_stage1_br, enc2_stage1_br, enc3_stage1_br = self.encoder1(x1_br)
        
        enc1_stage1_right, enc2_stage1_right, enc3_stage1_right = torch.cat((enc1_stage1_tr,enc1_stage1_br), dim = 2), torch.cat((enc2_stage1_tr,enc2_stage1_br), dim = 2), torch.cat((enc3_stage1_tr,enc3_stage1_br), dim = 2)
        
        dec1_stage1_left, dec2_stage1_left, dec3_stage1_left = self.decoder1(enc1_stage1_left, enc2_stage1_left, enc3_stage1_left)
        dec1_stage1_right, dec2_stage1_right, dec3_stage1_right = self.decoder1(enc1_stage1_right, enc2_stage1_right, enc3_stage1_right)
        
        f_out_stage1_left, x_s1_left = self.sam_stage1(dec1_stage1_left, left_image)
        f_out_stage1_right, x_s1_right = self.sam_stage1(dec1_stage1_right, right_image)
        
        x_s1 = torch.cat((x_s1_left, x_s1_right), dim = 3)
        
        x2_left = self.conv_stage2(left_image)
        x2_left = self.cab_stage2(x2_left)
        enc1_stage2_left, enc2_stage2_left, enc3_stage2_left = self.encoder2(x2_left, enc1_stage1_left, dec1_stage1_left, enc2_stage1_left, dec2_stage1_left, enc3_stage1_left, dec3_stage1_left)
        
        x2_right = self.conv_stage2(right_image)
        x2_right = self.cab_stage2(x2_right)
        enc1_stage2_right, enc2_stage2_right, enc3_stage2_right = self.encoder2(x2_right, enc1_stage1_right, dec1_stage1_right, enc2_stage1_right, dec2_stage1_right, enc3_stage1_right, dec3_stage1_right)
        
        enc1_stage2, enc2_stage2, enc3_stage2 = torch.cat((enc1_stage2_left, enc1_stage2_right), dim= 3), torch.cat((enc2_stage2_left, enc2_stage2_right), dim= 3), torch.cat((enc3_stage2_left, enc3_stage2_right), dim= 3)
        
        dec1_stage2, dec2_stage2, dec3_stage2 = self.decoder2(enc1_stage2, enc2_stage2, enc3_stage2)
        
        f_out_stage2, x_s2 = self.sam_stage2(dec1_stage2, image)
        
        x3 = self.conv_stage3(image)
        x3 = self.cab_stage3(x3)
        x3 = x3 + f_out_stage2
        x3 = self.ors_net(x3, enc1_stage2, dec1_stage2, enc2_stage2, dec2_stage2, enc3_stage2, dec3_stage2)
        x3 = self.final_conv(x3)
        x_s3 = x3 + image
        
        
        return x_s1, x_s2, x_s3
        
        
        
        

In [2]:
TRAIN_DIR = "/kaggle/input/a-curated-list-of-image-deblurring-datasets/DBlur/Gopro/train"
VAL_DIR = "/kaggle/input/a-curated-list-of-image-deblurring-datasets/DBlur/Gopro/test"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

import os
import random
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

to_tensor = transforms.ToTensor()

class GoProDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.blur_dir = os.path.join(root_dir, 'blur')
        self.sharp_dir = os.path.join(root_dir, 'sharp')
        self.image_files = os.listdir(self.blur_dir)
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        blur_image_path = os.path.join(self.blur_dir, self.image_files[idx])
        sharp_image_path = os.path.join(self.sharp_dir, self.image_files[idx])

        blur_image = Image.open(blur_image_path)
        sharp_image = Image.open(sharp_image_path)

        blur_image = to_tensor(blur_image)
        sharp_image = to_tensor(sharp_image)
            
        
        """hh, ww = tar_img.shape[1], tar_img.shape[2]

        rr     = random.randint(0, hh-ps)
        cc     = random.randint(0, ww-ps)
        aug    = random.randint(0, 8)

        # Crop patch
        inp_img = inp_img[:, rr:rr+ps, cc:cc+ps]
        tar_img = tar_img[:, rr:rr+ps, cc:cc+ps]"""
            
        patch_size = 256
        
        
        height = blur_image.shape[1]
        width = blur_image.shape[2]
        
        
        patch_start_y = random.randint(0, height - patch_size)
        patch_start_x = random.randint(0, width - patch_size)
        
        blur_image = blur_image[: , patch_start_y : patch_start_y + patch_size, patch_start_x : patch_start_x + patch_size]
        sharp_image = sharp_image[: , patch_start_y : patch_start_y + patch_size, patch_start_x : patch_start_x + patch_size]
        

        return blur_image, sharp_image
    
transform = transforms.Compose([
    transforms.ToTensor(),
    
])

train_dataset = GoProDataset(root_dir = TRAIN_DIR, transform=transform)


TrainLoader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=16, drop_last=False, pin_memory=True)

val_dataset = GoProDataset(root_dir = VAL_DIR)

ValidationLoader = DataLoader(val_dataset, batch_size = 2, shuffle = True, num_workers=8, drop_last=False, pin_memory=True)

Using device: cuda


/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [3]:
'''Code for these losses are taken directly from the authors of the paper'''

class CharbonnierLoss(nn.Module):
    """Charbonnier Loss (L1)"""

    def __init__(self, eps=1e-3):
        super(CharbonnierLoss, self).__init__()
        self.eps = eps

    def forward(self, x, y):
        diff = x - y
        # loss = torch.sum(torch.sqrt(diff * diff + self.eps))
        loss = torch.mean(torch.sqrt((diff * diff) + (self.eps*self.eps)))
        return loss

class EdgeLoss(nn.Module):
    def __init__(self):
        super(EdgeLoss, self).__init__()
        k = torch.Tensor([[.05, .25, .4, .25, .05]])
        self.kernel = torch.matmul(k.t(),k).unsqueeze(0).repeat(3,1,1,1)
        if torch.cuda.is_available():
            self.kernel = self.kernel.cuda()
        self.loss = CharbonnierLoss()

    def conv_gauss(self, img):
        n_channels, _, kw, kh = self.kernel.shape
        img = F.pad(img, (kw//2, kh//2, kw//2, kh//2), mode='replicate')
        return F.conv2d(img, self.kernel, groups=n_channels)

    def laplacian_kernel(self, current):
        filtered    = self.conv_gauss(current)    # filter
        down        = filtered[:,:,::2,::2]       # downsample
        new_filter  = torch.zeros_like(filtered)
        new_filter[:,:,::2,::2] = down*4          # upsample
        filtered    = self.conv_gauss(new_filter) # filter
        diff = current - filtered
        return diff

    def forward(self, x, y):
        loss = self.loss(self.laplacian_kernel(x), self.laplacian_kernel(y))
        return loss

        

# Train Loop

In [ ]:
from tqdm import tqdm
import random
import time

import torch

def compute_psnr(img1, img2, max_val=1.0):
    """
    Compute the PSNR (Peak Signal-to-Noise Ratio) between two images after normalizing them.
    
    Args:
    - img1 (torch.Tensor): The first image tensor.
    - img2 (torch.Tensor): The second image tensor.
    - max_val (float): The maximum possible pixel value of the images before normalization.
    
    Returns:
    - float: The PSNR value.
    """
    
    # Normalize the images to [0, 1]
    img1 = img1 / max_val
    img2 = img2 / max_val
    
    mse = torch.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    return 10 * torch.log10(1.0 / mse)



    

learning_rate = 2e-4
weight_decay = 1e-1
cycle = 8
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

#model = MPRNET().to(device)

char_loss = CharbonnierLoss()
edge_loss = EdgeLoss()

#optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay, betas=(0.9, 0.999),eps=1e-8)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=cycle, eta_min=1e-6)

train_losses = []
val_losses = []

from tqdm import tqdm

best_psnr = 0
best_epoch = 0

for epoch in range(start_epoch, 101):
    #epoch_start_time = time.time()
    epoch_loss = 0
    train_id = 1

    model.train()
    
    # Initialize tqdm progress bar
    pbar = tqdm(total=len(TrainLoader), desc=f"Epoch {epoch}")
    
    for i, (blur_image, sharp_image) in enumerate(TrainLoader, 0):

        # zero_grad
        for param in model.parameters():
            param.grad = None

        blur_image = blur_image.to(device)
        sharp_image = sharp_image.to(device)

        restored_images = model(blur_image)
 
        # Compute loss at each stage
        loss_char = sum([char_loss(restored_images[j],sharp_image) for j in range(len(restored_images))])
        loss_edge = sum([edge_loss(restored_images[j],sharp_image) for j in range(len(restored_images))])
        loss = (loss_char) + (0.05*loss_edge)
       
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

        # Update tqdm progress bar
        pbar.set_postfix(loss=epoch_loss/(i+1))
        pbar.update(1)

    #### Evaluation ####
    
    model.eval()
    psnr_val_rgb = []
    for ii, (blur_image, sharp_image) in enumerate(ValidationLoader, 0):
        blur_image = blur_image.to(device)
        sharp_image = sharp_image.to(device)

        with torch.no_grad():
            restored_images = model(blur_image)
        restored = restored_images[-1]

        for res, tar in zip(restored, sharp_image):
            psnr_val_rgb.append(compute_psnr(res, tar))

    psnr_val_rgb = torch.stack(psnr_val_rgb).mean().item()

    if psnr_val_rgb > best_psnr:
        best_psnr = psnr_val_rgb
        best_epoch = epoch
        torch.save({'epoch': epoch, 
                    'state_dict': model.state_dict(),
                    'optimizer' : optimizer.state_dict()
                    }, os.path.join("/kaggle/working/","model_best.pth"))

    # Update tqdm progress bar with PSNR
    pbar.set_postfix(loss=epoch_loss/len(TrainLoader), psnr=psnr_val_rgb)
    print("[epoch %d PSNR: %.4f --- best_epoch %d Best_PSNR %.4f]" % (epoch, psnr_val_rgb, best_epoch, best_psnr))

    

    scheduler.step()
    
    print("------------------------------------------------------------------")
    print("Epoch: {}\tLoss: {:.4f}\tLearningRate {:.6f}".format(epoch,  epoch_loss, scheduler.get_lr()[0]))
    print("------------------------------------------------------------------")

    

    # Close the tqdm progress bar
    pbar.close()


Using device: cuda


Epoch 16: 100%|██████████| 1052/1052 [12:57<00:00,  1.35it/s, loss=0.0921, psnr=26.6]


[epoch 16 PSNR: 26.6191 --- best_epoch 16 Best_PSNR 26.6191]
------------------------------------------------------------------
Epoch: 16	Loss: 96.9167	LearningRate 0.000192
------------------------------------------------------------------


Epoch 17: 100%|██████████| 1052/1052 [12:58<00:00,  1.35it/s, loss=0.091, psnr=26.8]


[epoch 17 PSNR: 26.7636 --- best_epoch 17 Best_PSNR 26.7636]
------------------------------------------------------------------
Epoch: 17	Loss: 95.7264	LearningRate 0.000171
------------------------------------------------------------------


...

Epoch 50: 100%|██████████| 1052/1052 [12:57<00:00,  1.35it/s, loss=0.0844, psnr=27.7]


[epoch 50 PSNR: 27.6612 --- best_epoch 47 Best_PSNR 28.0776]
------------------------------------------------------------------
Epoch: 50	Loss: 88.7729	LearningRate 0.000139
------------------------------------------------------------------


Epoch 51: 100%|██████████| 1052/1052 [12:58<00:00,  1.35it/s, loss=0.0832, psnr=27.6]


[epoch 51 PSNR: 27.6425 --- best_epoch 47 Best_PSNR 28.0776]
------------------------------------------------------------------
Epoch: 51	Loss: 87.5285	LearningRate 0.000101
------------------------------------------------------------------


Epoch 52:  17%|█▋        | 184/1052 [02:04<09:26,  1.53it/s, loss=0.0806]

In [7]:
def load_checkpoint(model, optimizer, filename):
    # Note: Input model & optimizer should be pre-defined. This routine only updates their states.
    start_epoch = 0
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        checkpoint = torch.load(filename)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
                  .format(filename, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(filename))

    return model, optimizer, start_epoch
# Define the path to your saved checkpoint
checkpoint_path = "/kaggle/working/model_best.pth"

# Load the checkpoint
model, optimizer, start_epoch = load_checkpoint(model, optimizer, checkpoint_path)


=> loading checkpoint '/kaggle/working/model_best.pth'
=> loaded checkpoint '/kaggle/working/model_best.pth' (epoch 16)
